In [61]:
import pandas as pd
pd.set_option('display.max_colwidth', 250)
pd.set_option('max_rows', 2500)
from wordcloud import WordCloud
import matplotlib.pylab as plt
import matplotlib.dates as mdates
from datetime import datetime
start_time0 = datetime.now()
import spacy
import numpy as np
import os
import re
import nltk
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import xgboost as xgb
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import FrenchStemmer
from nltk import NaiveBayesClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.utils import resample

In [62]:
stopwords=['reformedesretraites','réformedesretraites','retraiter','grevegenerale','reformedesretraites']

In [63]:
greve_twitter_origin=pd.read_csv('greve_twitter_final.csv', index_col=[0])

In [64]:
greve_twitter_origin.tweet_processed2=[x.replace("""'""",'') for x in greve_twitter_origin.tweet_processed2]
greve_twitter_origin.date = greve_twitter_origin.date.map(lambda x: str(x)[:-15])

In [66]:
index_neg=[31,33,44,46,55,59,61,74,78,108,119,121,130,136,141,149,154,162,168,175,180,186,
           187,188,193,199,219,220,240,267,274,276,279,284,289,292,303,312,323,328,330,339,341,
           346,347,348,349,361,363,396,403,404,417,440,441,442,447,451,462,465,466,477,487,488,
           491,493,496,497,2,20,24,25,28,504,516,521,525,532,536,540,544,545,552,554,557,558,
           563,576,579,585,591,592,595,599,602,634,658,659,660,661,675,681,685,688,690,699,709,
          711,715,722,723,733,736,746,747,753,756,762,763,764,766,783,788,798,801,803,805,810,
           813,816,826,827,828,829,831,839,850,854,856,861,878,889,890,893,900,902,906,908,910,
           916,929,930,937,940,953,956,958,966,970,973,976,982,988,997,998,1001,1003,1006,1007,
           1008,1010,1011,1030,1031,1026,1034,1036,1037,1038,1039,1040,1042,1047,1049,1051,1053,
          1056,1057,1058,1060,1063,1064,1065,1067,1068,1069,1072,1074,1077,1081,1083,1084,1087,
          1091,1096,1097,1098,1100,1103,1108,1109,1116,117,1125,1145,1153,1154,1156,1158,1161,
          1169,1175,1178,1180,1192,1193,1203,1205,1207,1208,1210,1214,1237,1262,1264,1266,1279,
          1280,1282,1290,1293,1298,1305,1314,1316,1343,1345,1350,1351,1352,1353,1354,1356,1357,
           1361,1368,1374,1375,1377,1379,1380,1389,1396,1398,1400,1404,1407,1408,1410,1415,1421,
           1423,1424,1427,1430,1431,1433,1434,1437,1441,1442,1445,1453,1458,1468,1474,1485,1486,
           1493,1494,1495,1500,1501,1507,1509,1512,1517,1522,1527,1528,1530,1543,1544,1550,1553,
          1564,1565,1568,1569,1570,1574,1576,1583,1584,1597,1598,1600,1601,1606,1612,1617,1621,
          1623,1626,1626,1628,1630,1631,1634,1637,1645,1648,1650,1657,1664,1648,1650,1657,1664,
          1666,1669,1672,1673,1674,1682,1683,1690,1694,1696,1698,1700,1701,1702,1704,1705,1706,
          1707,1708,1710,1716,1717,1721,1732,1733,1736,1739,1748,1758,1776,1778,1782,1784,1785,
          1794, 1797,1799,1814,1826,1832,1835,1838,1839,1840,1842,1846,1849,1854,1856,1882,1883,
           1887,1901,1906,1907,1914,1917,1918,1923,1924,1928,1932,1935,1936,1942,1944,1947,1953,
          1954,1955,1960,1961,1967,1968,1972,1975,1983,1984,1987,1990,1992,1997]
index_pos=[14,22,56,79,122,163,195,237,242,253,400,407,408,410,434,435,439,443,457,459,479,
          528,569,617,666,741,755,855,919,942,974,975,1027,1055,1167,1179,1465,1513,1520,1614,
          1817,1818,1819,1820]
greve_twitter_origin['polarity'].iloc[index_neg] = 'negatif'
greve_twitter_origin['polarity'].iloc[index_pos] = 'positif'


In [67]:
greve_twitter_origin['polarity'][292]

'negatif'

In [152]:
greve_twitter_train = greve_twitter_origin[0:2000]
greve_twitter_train['polarity'].value_counts()

negatif    1383
positif     617
Name: polarity, dtype: int64

In [154]:
df_majority = greve_twitter_train[greve_twitter_train.polarity=='negatif']
df_minority = greve_twitter_train[greve_twitter_train.polarity=='positif']
 
# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=1383,    # to match majority class
                                 random_state=123) # reproducible results
 
# Combine majority class with upsampled minority class
greve_twitter_train = pd.concat([df_majority, df_minority_upsampled])
 
# Display new class counts
greve_twitter_train.polarity.value_counts()

negatif    1383
positif    1383
Name: polarity, dtype: int64

In [155]:
count_vectorizer = CountVectorizer(analyzer='word') 
cv = count_vectorizer.fit(greve_twitter_train['tweet_processed2'])
pickle.dump(cv, open("cv1.pkl", "wb"))
cv_ = count_vectorizer.fit_transform(greve_twitter_train['tweet_processed2'])
cv1 = pickle.load(open("cv1.pkl", 'rb'))
cv1_new = CountVectorizer(vocabulary = cv1.vocabulary_)

In [156]:
X_train_logreg,X_test_logreg,y_train_logreg,y_test_logreg = train_test_split(cv_,
                                                greve_twitter_train['polarity'] , test_size=.2,
                                                 random_state=42)

In [157]:
logreg = LogisticRegression(C=1,penalty="l2")
logreg.fit(X_train_logreg,y_train_logreg)
prediction_logreg = logreg.predict(X_test_logreg)
print(classification_report(prediction_logreg,y_test_logreg))

              precision    recall  f1-score   support

     negatif       0.80      0.94      0.86       243
     positif       0.95      0.81      0.88       311

   micro avg       0.87      0.87      0.87       554
   macro avg       0.87      0.88      0.87       554
weighted avg       0.88      0.87      0.87       554



In [158]:
greve_twitter=pd.read_csv('greve_twitter_final.csv', index_col=[0])
X_cv2 = cv1_new.transform(greve_twitter['tweet_processed2'])
greve_twitter['polarity2']=logreg.predict(X_cv2)

In [159]:
print(accuracy_score(greve_twitter['polarity'],greve_twitter['polarity2']))
greve_twitter.loc[greve_twitter['polarity2']=='positif'].drop(columns=['tweet_processed','tweet_processed2','polarity']).head(30)
#greve_twitter.loc[greve_twitter['polarity']!=greve_twitter['polarity2']].drop(columns=['tweet_processed','tweet_processed2'])

,date,tweet,polarity2
6,2019-12-30 23:14:53+00:00,"Article de @Nice_Matin [2/2] ""Emmanuel Macron devrait réaffirmer lors de ses voeux aux Français ""l'ambition forte du #gouvernement"" et la sienne pour la #RéformeDesRetraites, un ""projet de progrès social qui corrige de nombreuses inégalités"", a ...",positif
7,2019-12-30 23:14:52+00:00,"Article de @Nice_Matin [Thread 1/2] @EmmanuelMacron devrait réaffirmer son ""ambition forte"" pour la #RéformeDesRetraites 30/12/2019",positif
9,2019-12-30 23:12:51+00:00,C'est très instructif pour mettre en perspective les mouvements sociaux de notre époque! #GreveGenerale #reformedesretraites,positif
10,2019-12-30 23:12:25+00:00,Non c’est légitime #reformedesretraites,positif
14,2019-12-30 22:48:34+00:00,Arrêtons de parler de grève contre la réforme des retraites. Il ne s’agit que de la grève contre la réforme des régimes spéciaux de la #RATP et de la #SNCF. Rien de plus. #ReformedesRetraites,positif
19,2019-12-30 22:34:19+00:00,"Une réforme de justice sociale, tout ça quoi... #greve31decembre #Retraites #reformedesretraites",positif
21,2019-12-30 22:28:44+00:00,@pm_cheung @Ernesto52944864 #GiletsJaunes #reformedesretraites @pablo_rotelli,positif
22,2019-12-30 22:07:39+00:00,"@AnasseKazib , pas vraiment ma tasse de thé mais force est de constater qu'il tient bien sa place dans la #reformedesretraites , qu'il maîtrise son sujet, qu'il défend le morceau et qu'il est plutôt bon pour déstabiliser l'intelligentsia macronis...",positif
23,2019-12-30 22:06:11+00:00,"Les syndicats les plus anti-#RéformeDesRetraites rivalisent de #FakeNews pour la lier au principe de capitalisation alors qu'elle consolide celui de répartition. Mais en plus ce sont eux, tel #FO, qui gèrent le plus gros régime par capitalisatº. ...",positif
25,2019-12-30 22:02:25+00:00,"Leur #reformedesretraites , c’est non! Tout le monde est concerné!",positif


In [ ]:
grevetwitter = greve_twitter.drop(columns='polarity2',inplace=True)
greve_twitter.to_csv('greve_twitter_final_classif.csv')